## DS4DS Final Task:

Upload your solution (training pipeline and final model ready for inference) as one archive file (.zip) to moodle at least three days prior to your exam appointments. Request an appointment at least two weeks in advance via email (oliver.wallscheid@uni-siegen.de). The latest exam date will be by end of September 2025.

In [ ]:
using Pkg
Pkg.activate("Project.toml")
Pkg.instantiate()
Pkg.status()

In [ ]:
using MAT
using PlotlyJS
using LaTeXStrings
using Serialization
using StatsBase
using Flux

include("utils.jl");

In [ ]:
# Load data
function load_data()
    println("Loading data...")
    data = matread("train_data.mat")
    U = data["U"]
    X = data["X"]
    measurement_series = data["measurement_series"]
    dt = data["dt"]
    println("Data loaded: $(size(U)) inputs, $(size(X)) states")
    U_engineered = feature_engg(U)
    check_feature_temperature_correlations(U_engineered, X)
    return U_engineered, X, measurement_series, dt
end

In [ ]:
include("utils.jl")
# Data preparation
function prepare_sequences(U_raw, X_raw, measurement_series, series_indices; max_length=8000)
    sequences = []

    #Extraction
    for series_idx in series_indices
        u_series = Float32.(extract(U_raw, measurement_series, series_idx))
        x_series = Float32.(extract(X_raw, measurement_series, series_idx))

        # Limiting length of sequences
        T = min(size(x_series, 1), max_length)

        sequence = Dict(
            "U" => u_series[1:T, :],     # [T × N] inputs
            "X" => x_series[1:T, :],     # [T × 2] ground truth
            "x0" => x_series[1, :],      # [2] initial state
            "T" => T,
            "series_idx" => series_idx
        )

        push!(sequences, sequence)
        println("Series $series_idx: $T timesteps")
    end

    return sequences
end


In [ ]:
# Feature engineering function
function feature_engg(U)
     """
    [N × 9] derived features
    1. Total current magnitude
    2. Current squared (I2R losses)
    3. Oil heat removal (rotor)
    4. Oil heat removal (stator)
    5. RPM
    6. Combined cooling effectiveness
    u3: Absolute value of torque of drive shaft
    u7: RMS Phase Current
    u9: ||Ud||
    u18: Oil Temp at Entry (rotor)
    u19: Oil Temp at Entry (stator)
    u20: Oil Temp at Exit A (rotor)
    u21: Oil Temp at Exit B (stator)
    """

    N = size(U, 1)
    U_engineered = zeros(Float32, N, 13)

    rpm = U[:, 1]           # RPM
    torque = U[:, 2]        # Torque
    i_d = U[:, 4]           # d-current
    i_q = U[:, 5]           # q-current
    u_d = U[:, 8]           # d-voltage
    u_q = U[:, 10]          # q-voltage

    oil_flow_rotor = U[:, 16]      # Rotor oil flow
    oil_flow_stator = U[:, 17]     # Stator oil flow
    oil_temp_entry_rotor = U[:, 18]    # Oil entry temp (rotor)
    oil_temp_entry_stator = U[:, 19]   # Oil entry temp (stator)
    oil_temp_exit_rotor = U[:, 20]     # Oil exit temp A (rotor)

    println("Engineering thermal features:")

    # 1. Total current magnitude
    U_engineered[:, 1] = sqrt.(i_d.^2 .+ i_q.^2)

    # 2. Current squared (proportional to resistive losses I2R)
    U_engineered[:, 2] = (i_d.^2 .+ i_q.^2) / 1000

    # 3. Oil heat removal rate (rotor)
    # Heat removal ∝ flow_rate × temperature_rise
    temp_rise_rotor = abs.(oil_temp_exit_rotor .- oil_temp_entry_rotor)
    U_engineered[:, 3] = oil_flow_rotor .* temp_rise_rotor

    # 4. Oil heat removal rate (stator)
    temp_rise_stator = abs.(oil_temp_exit_rotor .- oil_temp_entry_stator)
    U_engineered[:, 4] = oil_flow_stator .* temp_rise_stator

    # 5. RPM
    U_engineered[:, 5] = rpm / 100

    # 6. Combined cooling effectiveness (total heat removal capacity)
    U_engineered[:, 6] = U_engineered[:, 3] .+ U_engineered[:, 4]

    # raw features
    U_engineered[:, 7] = U[:, 3]
    U_engineered[:, 8] = U[:, 7]
    U_engineered[:, 9] = U[:, 9]
    U_engineered[:, 10] = U[:, 18]
    U_engineered[:, 11] = U[:, 19]
    U_engineered[:, 12] = U[:, 20]
    U_engineered[:, 13] = U[:, 21]

    # NaN and Inf
    U_engineered = replace!(U_engineered, NaN => 0.0f0)
    U_engineered = replace!(U_engineered, Inf => 0.0f0)
    U_engineered = replace!(U_engineered, -Inf => 0.0f0)

    println("Feature engineering completed: $(size(U, 2)) → $(size(U_engineered, 2)) features")

    feature_names = ["Total_Current", "Current_Squared", "Oil_Heat_Rotor", "Oil_Heat_Stator",
                "RPM", "Total_Cooling",
                "u1_RPM", "u2_Torque", "u3_Torque_Abs", "u4_Current_d", "u5_Current_q",
                "u6_Unknown", "u7_RMS_Phase_Current", "u8_Voltage_d", "u9_Voltage_Magnitude",
                "u10_Voltage_q", "u12_Unknown", "u13_Unknown",
                "u15_Unknown", "u16_Oil_Flow_Rotor", "u17_Oil_Flow_Stator",
                "u18_Oil_Temp_Entry_Rotor", "u19_Oil_Temp_Entry_Stator", "u20_Oil_Temp_Exit_Rotor",
                "u21_Oil_Temp_Exit_Stator"]

    println("\nEngineered feature ranges:")
    for i in 1:size(U_engineered, 2)
        min_val = minimum(U_engineered[:, i])
        max_val = maximum(U_engineered[:, i])
        println("$(feature_names[i]): $(round(min_val, digits=2)) to $(round(max_val, digits=2))")
    end

    return U_engineered
end
function check_feature_temperature_correlations(U_engineered, X)
    """sCorrelation"""

    feature_names = ["Total_Current", "Current_Squared", "Oil_Heat_Rotor", "Oil_Heat_Stator",
                "RPM", "Total_Cooling",
                "u1_RPM", "u2_Torque", "u3_Torque_Abs", "u4_Current_d", "u5_Current_q",
                "u6_Unknown", "u7_RMS_Phase_Current", "u8_Voltage_d", "u9_Voltage_Magnitude",
                "u10_Voltage_q", "u12_Unknown", "u13_Unknown",
                "u15_Unknown", "u16_Oil_Flow_Rotor", "u17_Oil_Flow_Stator",
                "u18_Oil_Temp_Entry_Rotor", "u19_Oil_Temp_Entry_Stator", "u20_Oil_Temp_Exit_Rotor",
                "u21_Oil_Temp_Exit_Stator"]


    println("\n=== Feature-Temperature Correlations ===")
    println("Feature Name        | Stator Temp | Rotor Temp")
    println("-------------------|-------------|------------")

    for i in 1:size(U_engineered, 2)
        stator_corr = cor(U_engineered[:, i], X[:, 1])
        rotor_corr = cor(U_engineered[:, i], X[:, 2])
        println("$(rpad(feature_names[i], 18)) | $(rpad(round(stator_corr, digits=3), 11)) | $(round(rotor_corr, digits=3))")
    end
end

In [ ]:
#new normalisation
function compute_normalization_stats(sequences)

    all_U = vcat([seq["U"] for seq in sequences]...)
    all_X = vcat([seq["X"] for seq in sequences]...)

    U_stats = (min=minimum(all_U, dims=1), max=maximum(all_U, dims=1))
    X_stats = (min=minimum(all_X, dims=1), max=maximum(all_X, dims=1))

    return U_stats, X_stats
end

function normalize_minmax(data, stats)
    return (data .- stats.min) ./ (stats.max .- stats.min .+ 1f-8)
end

function denormalize_minmax(data, stats)
    return data .* (stats.max .- stats.min .+ 1f-8) .+ stats.min
end

In [ ]:
function create_thermal_model(n_inputs=13, n_states=2, hidden_dims=[64, 32], dropout_rate=0.2)
    input_dim = n_states + n_inputs

    chain = Chain(
        Dense(input_dim, hidden_dims[1], tanh),
        Dropout(dropout_rate),
        [Dense(hidden_dims[i-1], hidden_dims[i], tanh) for i in 2:length(hidden_dims)]...,
        Dropout(dropout_rate),
        Dense(hidden_dims[end], n_states)
    )
    return chain
end

In [ ]:
# Debug code to check model creation
function model_debug(model)

    # Check model type and structure
    println("Model type: ", typeof(model))
    println("Model structure: ", model)

    # Check layers
    println("\nLayers:")
    for (i, layer) in enumerate(model.layers)
        println("Layer $i: ", typeof(layer), " - ", layer)
    end

    # Count parameters
    total_params = sum(length, Flux.params(model); init=0)
    println("\nTotal parameters: ", total_params)

    # Individual layer parameter counts
    println("\nParameter breakdown:")
    for (i, layer) in enumerate(model.layers)
        if layer isa Dense
            layer_params = sum(length, Flux.params(layer))
            println("Layer $i parameters: ", layer_params)
        else
            println("Layer $i parameters: 0")
        end
    end

    # Test a forward pass
    println("\nTesting forward pass:")
    test_input = randn(Float32, 15)  # inputs
    try
        output = model(test_input)
        println("Forward pass successful. Output shape: ", size(output))
        println("Output: ", output)
    catch e
        println("Forward pass failed: ", e)
    end
end

In [ ]:
function train_thermal_model!(model, opt_state, train_sequences, val_sequences, U_stats, X_stats; epochs=70, patience =15)
    println("Starting training:")
    println("Training sequences: $(length(train_sequences))")
    println("Validation sequences: $(length(val_sequences))")

    train_losses = []
    val_losses = []

    # early stopping
    best_val_loss = Inf
    patience_counter = 0
    best_model_state = deepcopy(Flux.state(model))

    for epoch in 1:epochs
        epoch_loss = 0.0

        for seq in train_sequences
            loss = train_step!(model, opt_state, seq, U_stats, X_stats)
            epoch_loss += loss
        end
        avg_train_loss = epoch_loss / length(train_sequences)

        # Validation
        val_mse, _ = evaluate_autoregressive_rollout(model, val_sequences, U_stats, X_stats)

        push!(train_losses, avg_train_loss)
        push!(val_losses, val_mse)

        if val_mse < best_val_loss
            best_val_loss = val_mse
            patience_counter = 0
            best_model_state = deepcopy(Flux.state(model))
        else
            patience_counter += 1
        end

        if epoch % 5 == 0 || epoch == 1
            println("Epoch $epoch: Train Loss = $(round(avg_train_loss, digits=4)), Val MSE = $(round(val_mse, digits=4))")
        end

        if patience_counter >= patience
            println("Early stopping at epoch $epoch")
            Flux.loadmodel!(model, best_model_state)  # Restore best model
            break
        end
    end

    return train_losses, val_losses
end

In [ ]:
function train_step!(model, opt_state, sequence, U_stats, X_stats)
    # Normalize data
    U_norm = normalize_minmax(sequence["U"], U_stats)
    X_norm = normalize_minmax(sequence["X"], X_stats)

    T = sequence["T"]

    # Loss
    loss, grads = Flux.withgradient(model) do m
        total_loss = 0.0

        x_current = X_norm[1, :]  #initial state

        # Autoregressive
        for t in 2:T
            # Current input (use u[t-1] to predict x[t])
            u_t = U_norm[t-1, :]
            model_input = vcat(x_current, u_t)

            # Target: true_x[t]
            x_target = X_norm[t, :]

            # Prediction
            x_pred = m(model_input)

            # Loss
            total_loss += mean(abs2, x_pred - x_target)

            # Set initial
            x_current = x_pred
        end

        return total_loss / (T-1) #loss per timestep
    end

    # Update parameters
    Flux.update!(opt_state, model, grads[1])

    return loss
end

In [ ]:
function evaluate_autoregressive_rollout(model, sequences, U_stats, X_stats)
    Flux.testmode!(model)
    total_mse = 0.0
    all_predictions = []

    for seq in sequences
        T = seq["T"]
        #U_clean = clean_data(seq["U"])
        U_clean = seq["U"]
        U_norm = normalize_minmax(U_clean, U_stats)
        X_true = seq["X"]   #not normalized or cleaned as it is ground truth

        # Initialize prediction array
        X_pred = zeros(Float32, T, 2)

        # Starting with true initial state
        X_pred[1, :] = seq["x0"]
        x_current = normalize_minmax(reshape(seq["x0"], 1, :), X_stats)[1, :]

        # Autoregressive
        for t in 2:T
            # Current
            u_t = U_norm[t-1, :]

            # Model input: [predicted_x[t-1], u[t-1]]
            model_input = vcat(x_current, u_t)

            # Predict next state
            x_next_norm = model(model_input)

            # Denormalize and store
            x_next = denormalize_minmax(reshape(x_next_norm, 1, :), X_stats)[1, :]
            X_pred[t, :] = x_next

            # Update current state for next prediction (keep normalized)
            x_current = x_next_norm
            """
            if t == 10
                println("=== Debug timestep $t ===")
                println("Model input: ", model_input)
                println("Normalized prediction: ", x_next_norm)
                println("X_stats for denorm: min=$(X_stats.min), max=$(X_stats.max)")
                println("Denormalized: ", x_next)
                println("True value: ", X_true[t, :])
            end
            """
        end

        # Calculate MSE wihtout norm
        mse_val = mean((X_pred - X_true).^2)
        total_mse += mse_val
        push!(all_predictions, X_pred)
    end

    avg_mse = total_mse / length(sequences)
    Flux.trainmode!(model)
    return avg_mse, all_predictions
end

In [ ]:
# Plotting
function plot_sample_predictions(true_temps, pred_temps)
T = size(true_temps, 1)
    timesteps = 1:T

    # Stator (col1)
    p1 = PlotlyJS.plot([
        PlotlyJS.scatter(
            x=timesteps,
            y=true_temps[:, 1],
            mode="lines",
            name="True Stator",
            line=attr(color="blue")
        ),
        PlotlyJS.scatter(
            x=timesteps,
            y=pred_temps[:, 1],
            mode="lines",
            name="Predicted Stator",
            line=attr(color="red", dash="dash")
        )
    ], Layout(
        title="Stator Temperature Comparison",
        xaxis_title="Time Step",
        yaxis_title="Temperature (°C)"
    ))

    # Rotor (col2)
    p2 = PlotlyJS.plot([
        PlotlyJS.scatter(
            x=timesteps,
            y=true_temps[:, 2],
            mode="lines",
            name="True Rotor",
            line=attr(color="green")
        ),
        PlotlyJS.scatter(
            x=timesteps,
            y=pred_temps[:, 2],
            mode="lines",
            name="Predicted Rotor",
            line=attr(color="orange", dash="dash")
        )
    ], Layout(
        title="Rotor Temperature Comparison",
        xaxis_title="Time Step",
        yaxis_title="Temperature (°C)"
    ))

    # Save plots as images
    PlotlyJS.savefig(p1, "stator_temperature_comparison.png")
    PlotlyJS.savefig(p2, "rotor_temperature_comparison.png")

    # Display
    display(p1)
    display(p2)

end

In [ ]:
#Clean the data
function clean_data(U, X)
    return U, X
end

In [ ]:
function main()
    # Load data
    U, X, measurement_series, dt = load_data()

    # Clean the data
    #U_clean, X_clean = clean_data(U, X)
    U_clean = U
    X_clean = X

    # Preparing sequences for training
    println("\nPreparing sequences for training:")
    train_sequences = prepare_sequences(U_clean, X_clean, measurement_series, [1, 2, 3, 4, 5, 6])
    val_sequences = prepare_sequences(U_clean, X_clean, measurement_series, [7, 8])
    test_sequences = prepare_sequences(U_clean, X_clean, measurement_series, [9, 10])

    # Norm stats
    U_stats, X_stats = compute_normalization_stats(train_sequences)
    println("X_stats min: ", X_stats.min)
    println("X_stats max: ", X_stats.max)

    # Create model
    println("\nCreating model:")
    model = create_thermal_model(13, 2, [64, 32], 0.2)
    n_params = sum(length, Flux.state(model))
    println("Model created with $n_params parameters")

    # Debug model
    model_debug(model)

    # Optimizer
    optimizer = ADAM(0.001)
    opt_state = Flux.setup(optimizer, model)

    # Train model
    println("\nTraining model:")
    train_losses, val_losses = train_thermal_model!(
        model, opt_state, train_sequences, val_sequences, U_stats, X_stats, epochs=100, patience = 15
    )

    # Final evaluation on test set
    println("\nFinal evaluation...")
    test_mse, test_predictions = evaluate_autoregressive_rollout(model, test_sequences, U_stats, X_stats)
    println("Final Test MSE: $(round(test_mse, digits=4))")

    # Save model and normalization parameters
    println("\nSaving model...")
    model_data = Dict(
        "model_params" => Flux.state(model),
        "U_stats" => U_stats,
        "X_stats" => X_stats,
        "model_architecture" => [64, 32],  # For reconstruction
        "train_losses" => train_losses,
        "val_losses" => val_losses,
        "test_mse" => test_mse
    )

    serialize("final_task_parameters", model_data)
    println("Model saved to 'final_task_parameters'")

    # Plot training curves
    #plot_training_curves(train_losses, val_losses)

    # Plot sample predictions
    plot_sample_predictions(test_sequences[1]["X"], test_predictions[1])

    return model, model_data
end

In [ ]:
model, model_data = main()


In [ ]:
using DelimitedFiles

files = readdir(".")
println("Files in the current directory:")
for file in files
    println(file)
end

### Saving your model for evaluation:

After you have finished your model, we will evaluate its performance on the test dataset. To do so we will need to be able to make a forward pass with your model and we will need you to give us your model parameters.

- fill out the function given in ```model_forward_pass.jl```
- store your parameters in a file named ```final_task_parameters```

```
# Example code for storing parameters:

if @isdefined parameters
    serialize("final_task_parameters", parameters)
end
```

You can use the code below on the training to check if your result can be read properly **(If you run into problems with this or if your forward pass needs extra inputs, please contact us)**:

In [ ]:
include("model_forward_pass.jl");
using .ModelForwardPass

In [ ]:
?ModelForwardPass.your_model_forward_pass

In [ ]:
parameters_evaluation = deserialize("final_task_parameters");

measurement_series_idx = 7
 data = matread("train_data.mat")
 U = data["U"]
 X = data["X"]
 measurement_series = data["measurement_series"]
 dt = data["dt"]
_u = extract(U, measurement_series, measurement_series_idx);
_target = extract(X, measurement_series, measurement_series_idx);
_x0 = _target[1, :];

tsteps = collect(0:0.5:(size(_u, 1) * 0.5));

predicted_trajectory = ModelForwardPass.your_model_forward_pass(
    inputs=_u,
    x0=_x0,
    parameters=parameters_evaluation,
    tsteps=tsteps
);

println("MSE: ", StatsBase.mean((_target - predicted_trajectory).^2))

plot_sample_predictions(_target, predicted_trajectory)